# Identification de la paraphrase

## Problématique 
Il s'agit de dire si deux phrases partagent le même sens, ou autrement dit si elles sont paraphrases l'une de l'autre.


In [1]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Installation de l'env
import os
os.chdir("/content/drive/MyDrive/Projets Mars 2023 Mosef/ML AVANCE/Notebooks")

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from scripts.data_utils import clean_sequences_pair,collect_data_from_hugging_face,processing_pipeline,split_pairs_data,most_common_words_with_frequency

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Importation des données de HuggingFace

### Phase de processing
- Preprocessing: standardisation
- Potentielle vectorisation si utilisation d'un embedding Keras: L'entrée (input) d'un embedding Keras doit être une séquence d'entiers représentant les indices des mots dans un vocabulaire donné. Chaque entier représente l'indice d'un mot unique dans le vocabulaire. Les indices sont utilisés pour indexer une matrice d'embedding qui contient des représentations vectorielles de chaque mot dans le vocabulaire.


### Phase d'Embedding

#### Word2Vec

#### Doc2Vec

#### Transformers


In [6]:
import io
import os
import re
import shutil
import string

SOit on utilise doc2vec sinon on fait la moyenne des vecteurs de mots pour chaque phrase
L'input du modele doit être dans tous les cas un vecteur de dimension 

In [7]:
import pandas as pd
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
train_sentences_1, train_sentences_2, train_labels = processing_pipeline(train,True)
test_sentences_1, test_sentences_2, test_labels = processing_pipeline(test,True)

In [10]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')



In [12]:


dict_inputs = {"sentences_1":train_sentences_1,"sentences_2":train_sentences_2,"label":train_labels

}

In [13]:
dict_test_inputs = {"sentences_1":test_sentences_1,"sentences_2":test_sentences_2,"label":test_labels

}

In [14]:
from datasets import Dataset, load_metric

train_dataset = Dataset.from_dict(dict_inputs)


In [15]:
def encode(examples):
    return tokenizer(examples['sentences_1'], examples['sentences_2'],max_length=128, padding=True)

# Encoder les exemples d'entraînement et de validation
train_dataset = train_dataset.map(encode, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2364: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [16]:
test_dataset = Dataset.from_dict(dict_test_inputs)
test_dataset = test_dataset.map(encode)

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [17]:
test_dataset

Dataset({
    features: ['sentences_1', 'sentences_2', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 390965
})

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,BertForSequenceClassification
from datasets import load_metric
from transformers import DataCollatorWithPadding

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

import torch
import torch.nn.functional as F
metric = load_metric("accuracy")

def contrastive_loss(y, d):
    margin = 1
    return torch.mean(y * torch.square(d) + (1 - y) * torch.square(torch.clamp(margin - d, min=0)))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2, # Augmenter le nombre d'époques
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    learning_rate=5e-5, # Changer le learning rate
    save_total_limit=3, # Limiter le nombre de points de contrôle sauvegardés
    metric_for_best_model='accuracy')
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

# Créer un objet Trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model.cuda()  # add this line to move the model to GPU

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,    
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
trainer.evaluate()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

ValueError: ignored

In [22]:
titi =train_dataset["input_ids"]

In [24]:
[len(elem) for elem in titi]

[91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
 91,
